In [1]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import evaluate
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TrainingArguments,Trainer
from datasets import Dataset,DatasetDict
import numpy as np 
import pandas as pd 
import kaggle, zipfile
import warnings

# Código para ignorar avisos de usuário
warnings.filterwarnings("ignore", category=UserWarning)

# Artigo 4 - Natural Language Processing

In [2]:
path = 'nlp-getting-started'
#kaggle.api.competition_download_cli(str(path))
#zipfile.ZipFile(f'{path}.zip').extractall(path)

Utilizar um `!` antes do comando o faz ser executado no terminal, com isso podemos checar o conteúdo da pasta que foi baixada:

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


Vamos criar a variável que vai guardar o nosso DataFrame de treino:

In [4]:
df = pd.read_csv(path + '/train.csv')

Usamos o atributo `head()` para ver as 5 primeiras linhas da tabela:

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Então nossas colunas são `id | keyword | location | text | target`.  
Podemos usar a seguinte função para ver um exemplo de um tweet que não é sobre um desastre:

In [6]:
df[df["target"] == 0]["text"].values[1]

'I love fruits'

`df["target"] == 0` garante que será pego um valor cuja coluna `target`, que indica se o tweet é ou não sobre um desastre, seja 0, ou seja, falso.

In [7]:
df.describe(include='object')

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


In [8]:
# train_df['input'] = 'TWT: ' + train_df.text + '; KWD: ' + str(train_df.keyword) + '; LOC: ' + str(train_df.location)

In [9]:
# train_df[train_df["target"] == 0]['input'].values[1]

In [10]:
df.drop(columns = ['id', 'keyword', 'location'], inplace = True)
df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’

In [11]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'target'],
    num_rows: 7613
})

In [12]:
model_nm = 'microsoft/deberta-v3-small'
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
tokz.tokenize("According to all known laws of aviation, ...")

['▁According',
 '▁to',
 '▁all',
 '▁known',
 '▁laws',
 '▁of',
 '▁aviation',
 ',',
 '▁.',
 '.',
 '.']

In [14]:
def tok_func(x): return tokz(x["text"])
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [15]:
row = tok_ds[0]
row['text'], row['input_ids']

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 [1,
  581,
  65453,
  281,
  262,
  18037,
  265,
  291,
  953,
  117831,
  903,
  4924,
  17018,
  43632,
  381,
  305,
  2])

In [16]:
tokz.vocab['▁are']

281

In [17]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

## Conjuntos de teste e validação

### Validação

Para evitar *overfitting*, que é quando uma função fica ajustada demais ao conjunto de dados em que foi treinada e acaba desviando do formato real da função que estava sendo aproximada, é importante separar um conjunto de dados para não incluir no treinamento, pois não queremos que o modelo o veja.

Isso é chamado de *conjunto de validação*, e a biblioteca Transformers disponibiliza o `train_test_split()` para sua criação:

In [18]:
dds = tok_ds.train_test_split(0.25, seed=42) # 25% para validação e 75% para treino
dds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5709
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1904
    })
})

Aqui o conjunto de validação é chamado `test`.

### Teste

O *conjunto de teste* também é isento do treino, e só é usado após concluído todo o processo de treinamento.

A medida em que você tenta coisas diferentes para ver o impacto nas métricas do teste de validação, é possível que encontre algumas coisas que acidentalmente melhorem os resultados naquela situação, mas que no mundo real não são exatamente melhores. Por isso o conjunto de teste serve para mostrar se ocorreu *overfitting* com o conjunto de validação.

In [19]:
eval_df = pd.read_csv(path + '/test.csv')
eval_df.drop(columns = ['id', 'keyword', 'location'], inplace = True)
eval_df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

Chamamos de `eval` para não confundir com o outro conjunto criado acima.

In [20]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    return f1.compute(predictions=eval_pred)

## Treinando o modelo

In [21]:
bs = 64
epochs = 4
lr = 8e-5

In [22]:
args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True, 
    evaluation_strategy="epoch", 
    per_device_train_batch_size=bs, 
    per_device_eval_batch_size=bs*2, 
    num_train_epochs=epochs, 
    weight_decay=0.01, 
    report_to='none'
    )

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(
    model, 
    args, 
    train_dataset=dds['train'], 
    eval_dataset=dds['test'],
    tokenizer=tokz, 
    )

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [24]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/mines/mambaforge/envs/apm-env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5709
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 360
  Number of trainable parameters = 141895681
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__cal

Epoch,Training Loss,Validation Loss
1,No log,0.125910
2,No log,0.130627
3,No log,0.129852
4,No log,0.146057


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904

In [34]:
predictions = trainer.predict(dds['test'])

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1904
  Batch size = 128


In [37]:
preds = np.clip(predictions.predictions, 0, 1)
print(predictions.label_ids)
print(preds)

[1. 0. 1. ... 1. 1. 0.]
[1.         0.11853027 0.8852539  ... 0.9580078  0.07342529 0.0166626 ]


In [38]:
metric = evaluate.load('f1')
metric.compute(predictions=preds, references=predictions.label_ids)

{'f1': 0.7550143266475645}

In [39]:
def compute_metrics(eval_preds):
    metric = evaluate.load('f1')
    logits, labels = eval_preds
    predictions = np.clip(logits, 0, 1)
    return metric.compute(predictions=predictions, references=labels)